# ID2214/FID3214 Assignment 2 Group no. [20]
### Project members: 
[Paddy flanagan, paddyfl@kth.se]
[HUAI-I TUNG, hitung@kth.se]
[Vasigaran Senthilkumar, vsen@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [1]:

import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def accuracy(dataframe, correctlabels):
    
    df = dataframe.copy()
    labels = df.idxmax(axis=1)
    truelabels = (labels == correctlabels).sum(axis=0)
    accuracy = truelabels/len(df)
    return accuracy


def brier_score(df, correctlabels):
    score = 0  
    observed_probs = pd.get_dummies(correctlabels)
    score = (df - observed_probs) ** 2
    score = score.sum(axis=1)
    score = score.mean()
        
    return score

def calc_auc(predictions,correctlabels,col):
    pos, neg =get_pos_and_neg(predictions,correctlabels,col)
    pos=pos.tolist()
    neg=neg.tolist()
    tot_tp=0
    tot_fp=0
    for i in range(len(pos)):
        tot_tp=pos[i]+tot_tp
        tot_fp=tot_fp+neg[i]
        
    a=0
    cov_tp=0
    for i in range(len(pos)):
        if neg[i]==0 :
            cov_tp=cov_tp+pos[i]
        elif pos[i]==0 :
            a=a+(cov_tp/tot_tp)*(neg[i]/tot_fp)
        else:
            a=a+(cov_tp/tot_tp)*(neg[i]/tot_fp)+(pos[i]/tot_tp)*(neg[i]/tot_fp)/2
            cov_tp=cov_tp+pos[i]
    return a

def auc(predictions,correctlabels):
    class_frequency = dict(pd.Series(correctlabels).value_counts(normalize = True))
    x=0
    
    for col in predictions.columns:
        x=x+calc_auc(predictions,correctlabels,col)*class_frequency[col]
    return x 

def get_pos_and_neg(predictions,correctlabels,pos_label):
    scores={}
    
    for i in range(len(predictions)):
        score = predictions.loc[i,pos_label]
        (p,n) = scores.get(score,(0,0))
        if correctlabels[i] == pos_label:
            scores[score] = (p+1,n)
        else:
            scores[score] = (p,n+1)
    scores = np.array([[score,scores[score][0],scores[score][1]] for score in scores.keys()])
    #display(scores)
    scores = scores[np.argsort(scores[:,0])[::-1]]
    pos = scores[:,1]
    neg = scores[:,2]
    #display(pos,neg)
    return pos, neg

def create_bins(glass_train_df,nobins,bintype):
    df2=glass_train_df.copy()
    binning=dict()
    for col in df2.columns:
        if(col not in['CLASS','ID'] and df2.dtypes[col] in ['float64','int64','float32','int32']):
            if(bintype=='equal-size'):
                df2[col],bins=pd.qcut(df2[col],q=10,retbins=True,labels=False,duplicates='drop')
            elif(bintype=='equal-width'):
                df2[col],bins=pd.cut(df2[col],10,retbins=True,labels=False,duplicates='drop')
            df2[col]=df2[col].astype('category')
            df2[col]=df2[col].cat.set_categories([str(i) for i in df2[col].cat.categories],rename=True)
            bins[0]=-np.inf
            bins[-1]=np.inf
            binning[col]=bins
    return df2,binning

def apply_bins(dataframe,binning):
    
    df = dataframe.copy()
    bin_labels = {}
    for col in binning:  
        bins = binning[col]
        df[col] = pd.cut(df[col],bins,labels=False)
        df[col] = df[col].astype("category")
        df[col] = df[col].cat.set_categories([str(i) for i in df[col].cat.categories], rename = True)        
    df = df.astype("category")
    return df

def create_column_filter(df):
    df1=df.copy()
    #display(df1)
    rem=list()
    for col in df1.columns:
        if col not in ['CLASS','ID'] and df1[col].nunique()<=1:
        
            df1=df1.drop(columns=col)
        else:
            rem.append(col)
    return df1,rem 

def create_imputation(dataframe):
    df = dataframe.copy()
    imputation = {}
    
    #select numerical and categorical columns
    numeric_column = df.select_dtypes(include=["int64", "float64"]).columns
    categorical_column = df.select_dtypes(exclude=["int64", "float64"]).columns
    #exclude class and id column 
    numeric_column = [column for column in numeric_column if column != "ID" if column != "CLASS"]
    categorical_column = [column for column in categorical_column if column != "ID" if column != "CLASS"]
    
    for column in df.columns:
        if (column not in ['CLASS','ID'] and df.dtypes[column] in ['float64','int64','float32','int32']):
            if df[column].any():
                imputation[column] = df[column].mean()
                df[column]=df[column].fillna(df[column].mean())
                
            #Replace whole column with Nan to 0
            else:
                df[column]=df[column].fillna(0)
                imputation[column] = 0
                
        elif (column not in ['CLASS','ID'] and df.dtypes[column] in ['object','category']):
            if df[column].any():
                imputation[column] = df[column].mode()[0]
                df[column]=df[column].fillna(df[column].mode()[0])
                
            #Replace whole column with Nan to ""
            else:
                imputation[column] = ""
                df[column]=df[column].fillna("")
                
    
    return df, imputation

def create_one_hot(train_df):
    
    df1=train_df.copy()
    d=dict()
    
    for col in df1.select_dtypes(include=['category','object']).columns:
        if(col not in ['CLASS','ID'] ):
            
            
            x=pd.get_dummies(df1[col],prefix=col)
            x=x.astype('float')
            d[col]=df1[col].unique().tolist()
            df1=df1.drop(columns=col)
            df1=pd.concat([df1,x],axis=1,join='inner')
            
    return df1,d

def create_normalization(df,normalizationtype='minmax'):
    new_df=df.copy()
    
    colss = new_df.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    colss = colss.select_dtypes(include=[np.int_, np.float_]) 
    
    cols=colss.columns.tolist()

    if normalizationtype=='minmax':
        mins=new_df[cols].min()
        maxs=new_df[cols].max()
        types=[normalizationtype]*len(cols)
        minmax=zip(types,mins,maxs)
        normalization=dict(zip(cols,minmax))
        new_df[cols]=(new_df[cols]-mins)/(maxs-mins)
    
    if normalizationtype=='zscore':
        means=new_df[cols].mean()
        stds=new_df[cols].std()
        types=[normalizationtype]*len(cols)
        zscore=zip(types,means,stds)
        normalization=dict(zip(cols,zscore))
        new_df[cols]=(new_df[cols]-means)/(stds)   
    
    return new_df,normalization

def apply_imputation(dataframe,imputation):
    df = dataframe.copy()
    
    for column in imputation.keys():
        if column not in ['CLASS','ID'] :
            df[column]=df[column].fillna(imputation[column])
    
    return df

def apply_normalization(df, normalization):
    new_df = df.copy()
    cols = new_df.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    cols = cols.select_dtypes(include=[np.int_, np.float_])
    
    colss = cols.columns.tolist()
    
    for col in colss:
        if normalization[col][0] == "minmax":
            new_df[col] = np.clip([(x - normalization[col][1])/(normalization[col][2]-normalization[col][1]) for x in df[col]], 0, 1) # limit the output range to 0,1
            
        elif normalization[col][0] == "zscore":
            new_df[col] = [(x - normalization[col][1])/normalization[col][2] for x in df[col]]
    
    return new_df

def apply_column_filter(df,column_filter):
    df1=df.copy()
    df1=df1[df1.columns.intersection(column_filter)]
    return df1

## 1. Define the class kNN

In [89]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies


class kNN:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None
        
    def fit(self, df, normalizationtype= "minmax"):
        
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype)
        #df, self.one_hot = create_one_hot(df)
        df["CLASS"] = df["CLASS"].astype("category")
        self.training_labels = df["CLASS"]
        self.labels = list(df["CLASS"].cat.categories)
        self.training_data = df.drop(labels = ['CLASS', 'ID'], axis = 1)
        
        
    def get_nearest_neighbor_predictions(self, x_test, k):
        neighbours = []
        distances = []  
        data = pd.DataFrame(self.training_data)
        
        num_rows = data.shape[0]
        
        for i in range(num_rows):
            dist = 0.0
            values = np.array(data.iloc[i,:].values)
            dist = self.euclidean_distance(values, x_test)
            distances.append((i, dist))
            
        distances.sort(key=lambda tup: tup[1])
    
        for j in range(k):
            neighbours.append(distances[j][0])
            
        return neighbours  

    
        
    def euclidean_distance(self, row1, row2):
        return np.sqrt(np.sum(np.power(row1 - row2, 2), axis=0))
        
        
    def get_prob(self, k_NN_indexes):
        labels = self.labels
        k = len(k_NN_indexes)
        labels_prob = np.zeros(len(labels))
        
        for i in range(k):
            for j in range(len(labels)):
                if self.training_labels[k_NN_indexes[i]] == labels[j]:
                    labels_prob[j] += 1
        prob = labels_prob/k
        return prob
    
    def predict(self, dataframe, k):

        df = dataframe.copy()
        df.drop(columns=["ID", "CLASS"], inplace=True)
        
        df = apply_column_filter(df, self.column_filter)
        df = apply_normalization(df, self.normalization)
        df = apply_imputation(df, self.imputation)
        #df = apply_one_hot(df, self.one_hot)
        num_rows = df.shape[0]
        s = (num_rows, len(self.labels))
        predictions = np.zeros(s)
        
        for i in range(num_rows):
            values = np.array(df.iloc[i,:].values)
            rowindex = self.get_nearest_neighbor_predictions(values, k)
            prob = self.get_prob(rowindex)
            predictions[i] = prob
        
        predictions_df = pd.DataFrame(predictions, columns=self.labels)
        return predictions_df
        
        
        
        
        
        
        
    
        
        


In [90]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.02 s.
Testing time (k=1): 1.60 s.
Testing time (k=3): 1.56 s.
Testing time (k=5): 1.67 s.
Testing time (k=7): 1.68 s.
Testing time (k=9): 1.83 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [91]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [13]:
import numpy as np
import pandas as pd
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
class NaiveBayes():
    def __init__(self):
        self.column_filter=None
        self.binning=None
        self.labels=None
        self.class_priors=None
        self.feature_class_value_counts=None
        self.feature_class_counts=None
    
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>

    def fit(self,df,nobins=3,bintype="equal-size"):
        df1=df.copy()
        df1,self.column_filter=create_column_filter(df1)
        df1, self.binning = create_bins(df1,nobins,bintype)
        df1['CLASS']=df1['CLASS'].astype('category')
        self.labels=list(df1['CLASS'].cat.categories)
        self.class_priors=dict((pd.Series(df['CLASS']).value_counts(normalize=True)))
        self.feature_class_value_counts=dict()
        j=dict()
        for col in df1.columns :
            if(col not in['CLASS','ID']):
                df2=df1.dropna(axis=0,how='any',subset=['CLASS',col])
                j[col]=dict((pd.Series(df['CLASS']).value_counts()))
                i=df2.groupby([col,'CLASS']).size()
                self.feature_class_value_counts[col]=dict(i)
        self.feature_class_counts=j
        #display(self.feature_class_counts)
        #display(self.feature_class_value_counts)
        
    
    

#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#

    def predict(self,df):
        df1=df.copy()
        df1=apply_column_filter(df1,self.column_filter)
        df1 = apply_bins(df1,self.binning)
        
        df1=df1.drop(columns=['ID','CLASS'])
        coln=df1.shape[1]
        rown=df1.shape[0]
        labeln=len(self.labels)
        mat=np.zeros([labeln,rown,coln])
        temp=dict()
        #display(self.feature_class_value_counts)
        #for i in range(rown):
        for k in self.labels:
            u=dict()
            for i in range(coln):
                col=df1.columns[i]
                #display(col)
        
                f=dict()
                for j in range(rown):
                    val=df1.iloc[j,i]
                    #display(val)
                    if((val,k) in self.feature_class_value_counts[col].keys()):
                        num=self.feature_class_value_counts[col][(val,k)]
                        #display(num)
                        den=self.feature_class_counts[col][k]
                        rfq=num/den
                        
                    else:
                        rfq=0
                    
                    f[val]=rfq
                
                u[col]=f
            temp[k]=u
                    
                
        display(temp)            
            

        
                    
        
                    
            




            
    
    
        
    
    

# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors



In [14]:
glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()
nb_model.fit(glass_train_df)
nb_model.predict(glass_test_df)

{'RI': {('0', 1): 4,
  ('0', 2): 1,
  ('0', 3): 0,
  ('0', 5): 2,
  ('0', 6): 0,
  ('0', 7): 4,
  ('1', 1): 2,
  ('1', 2): 6,
  ('1', 3): 1,
  ('1', 5): 0,
  ('1', 6): 0,
  ('1', 7): 3,
  ('2', 1): 0,
  ('2', 2): 4,
  ('2', 3): 1,
  ('2', 5): 0,
  ('2', 6): 0,
  ('2', 7): 4,
  ('3', 1): 0,
  ('3', 2): 6,
  ('3', 3): 0,
  ('3', 5): 1,
  ('3', 6): 0,
  ('3', 7): 4,
  ('4', 1): 4,
  ('4', 2): 5,
  ('4', 3): 0,
  ('4', 5): 0,
  ('4', 6): 0,
  ('4', 7): 2,
  ('5', 1): 8,
  ('5', 2): 0,
  ('5', 3): 2,
  ('5', 5): 0,
  ('5', 6): 0,
  ('5', 7): 0,
  ('6', 1): 2,
  ('6', 2): 6,
  ('6', 3): 2,
  ('6', 5): 0,
  ('6', 6): 1,
  ('6', 7): 0,
  ('7', 1): 4,
  ('7', 2): 0,
  ('7', 3): 0,
  ('7', 5): 1,
  ('7', 6): 5,
  ('7', 7): 0,
  ('8', 1): 4,
  ('8', 2): 2,
  ('8', 3): 1,
  ('8', 5): 4,
  ('8', 6): 0,
  ('8', 7): 0,
  ('9', 1): 3,
  ('9', 2): 4,
  ('9', 3): 1,
  ('9', 5): 0,
  ('9', 6): 0,
  ('9', 7): 3},
 'Na': {('0', 1): 3,
  ('0', 2): 5,
  ('0', 3): 0,
  ('0', 5): 3,
  ('0', 6): 0,
  ('0', 7): 

{1: {'RI': {'3': 0.0,
   '9': 0.0967741935483871,
   '5': 0.25806451612903225,
   '1': 0.06451612903225806,
   '6': 0.06451612903225806,
   '4': 0.12903225806451613,
   '0': 0.12903225806451613,
   '8': 0.12903225806451613,
   '2': 0.0,
   '7': 0.12903225806451613},
  'Na': {'1': 0.1935483870967742,
   '7': 0.0967741935483871,
   '6': 0.12903225806451613,
   '0': 0.0967741935483871,
   '3': 0.16129032258064516,
   '4': 0.06451612903225806,
   '5': 0.12903225806451613,
   '9': 0.0,
   '8': 0.06451612903225806,
   '2': 0.06451612903225806},
  'Mg': {'1': 0.06451612903225806,
   '7': 0.1935483870967742,
   '6': 0.16129032258064516,
   '4': 0.1935483870967742,
   '5': 0.0967741935483871,
   '0': 0.0,
   '3': 0.1935483870967742,
   '2': 0.0967741935483871},
  'Al': {'4': 0.0967741935483871,
   '0': 0.22580645161290322,
   '1': 0.1935483870967742,
   '9': 0.0,
   '2': 0.22580645161290322,
   '8': 0.0,
   '3': 0.12903225806451613,
   '5': 0.06451612903225806,
   '7': 0.0,
   '6': 0.0645161290

In [80]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.12 s.
Testing time (3, 'equal-width'): 0.25 s.
Training time (3, 'equal-size'): 0.11 s.
Testing time (3, 'equal-size'): 0.28 s.
Training time (5, 'equal-width'): 0.12 s.
Testing time (5, 'equal-width'): 0.28 s.
Training time (5, 'equal-size'): 0.11 s.
Testing time (5, 'equal-size'): 0.26 s.
Training time (10, 'equal-width'): 0.10 s.
Testing time (10, 'equal-width'): 0.24 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.23 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.598131     0.570270  0.747255
   equal-size   0.579439     0.743837  0.746409
5  equal-width  0.598131     0.570270  0.747255
   equal-size   0.579439     0.743837  0.746409
10 equal-width  0.598131     0.570270  0.747255
   equal-size   0.579439     0.743837  0.746409

In [81]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8972
AUC on training set: 0.9906
Brier score on training set: 0.1461


### Comment on assumptions, things that do not work properly, etc.